In [ ]:
import pandas as pd
import requests

# URL da página com as tabelas
url = 'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/245234'

# Baixar o conteúdo da página
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    tabelas = pd.read_html(response.text, header=1)  # Define a segunda linha como cabeçalho

    # Dicionário para armazenar os DataFrames
    dfs = {}

    # Criar um DataFrame para cada tabela e armazenar no dicionário
    for i, tabela in enumerate(tabelas):
        nome_df = f'df{i+1}'
        dfs[nome_df] = tabela

else:
    print(f"Erro ao acessar a página: {response.status_code}")

ModuleNotFoundError: No module named 'pandas'

#### Como funciona cada coluna ?

## Descrição das Colunas

- **MÊS**: Número do mês (1 a 12).
- **Grupo Estrutural**: Numero de passageiros transportados em onibus do tipo de linha (ex: principal, alimentadora dos centros e afins).
- **Grupo Local de Articulação Regional**: Numero de passageiros transportados em linhas regionais
- **Grupo Local de Distribuição**: Numero de passageiros transportados em linhas regionais

- **Total**: Total de passageiros ou unidades transportadas.
- **particao**: Segmentação dos dados, geralmente por ano ou período.
- **Período**: Intervalo de tempo de coleta (ex: 2024).

In [ ]:
for i in range(1, 13, 1):
  print('mundo')
  df = dfs[f'df{i}']
  if i == 1:
    valor = 2024
  else:
    valor = valor - 1
  df['particao'] = valor
  if i == 1:
    df_final = df.copy()
  else:
    # Renomeando e substituindo as colunas com os valores e apagando as antigas
    if 'Estrutural' in df.columns:
        df['Grupo Estrutural'] = df['Estrutural']
        df.drop(columns=['Estrutural'], inplace=True)

    if 'Local' in df.columns:
        df['Grupo Local de Articulação Regional'] = df['Local']
        df.drop(columns=['Local'], inplace=True)

    if 'Sistema' in df.columns:
        df['Total'] = df['Sistema']
        df.drop(columns=['Sistema'], inplace=True)

    if 'Subsistema Estrutural' in df.columns:
        df['Grupo Estrutural'] = df['Subsistema Estrutural']
        df.drop(columns=['Subsistema Estrutural'], inplace=True)

    if 'Subsistema Local' in df.columns:
        df['Grupo Local de Articulação Regional'] = df['Subsistema Local']
        df.drop(columns=['Subsistema Local'], inplace=True)
        # Adiciona o DataFrame na lista

          # Verifica se as colunas existem
        # Verifica se a coluna 'Grupo Local de Distribuição' existe
    if 'Grupo Local de Distribuição' in df.columns:
        print('entrou')
        # Substituindo valores ausentes (NaN) por 0, mas não mexendo em valores válidos
        df['Grupo Local de Distribuição'] = df['Grupo Local de Distribuição'].fillna(0)
        df['Grupo Local de Articulação Regional'] = df['Grupo Local de Articulação Regional'].fillna(0)


        # Removendo os pontos (separadores de milhar) e tratando o "-" como 0
        df['Grupo Local de Articulação Regional'] = df['Grupo Local de Articulação Regional'].replace({'.': '', '-': '0'}, regex=True)
        df['Grupo Local de Distribuição'] = df['Grupo Local de Distribuição'].replace({'.': '', '-': '0'}, regex=True)

        # Removendo a coluna 'Grupo Local de Distribuição' já que foi somada
        df.drop(columns=['Grupo Local de Distribuição'], inplace=True)


    # Decrementa o valor da partição
    valor -= 1
    df_final = pd.concat([df_final, df])

In [ ]:
# Remove os pontos (separadores de milhar)
df_final['Grupo Local de Distribuição'] = df_final['Grupo Local de Distribuição'].str.replace('.', '', regex=False)
df_final['Grupo Local de Articulação Regional'] = df_final['Grupo Local de Articulação Regional'].str.replace('.', '', regex=False)
df_final['Total'] = df_final['Total'].str.replace('.', '', regex=False)
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].str.replace('.', '', regex=False)
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].str.replace('-', '', regex=False)




# Substitui valores ausentes (NaN) e strings vazias ('') por '0'
df_final['Grupo Local de Articulação Regional'] = df_final['Grupo Local de Articulação Regional'].fillna('0').replace('', '0')
df_final['Grupo Local de Distribuição'] = df_final['Grupo Local de Distribuição'].fillna('0').replace('', '0')
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].fillna('0').replace('', '0')

# Tenta converter as colunas para tipo float
df_final['Grupo Local de Articulação Regional'] = pd.to_numeric(df_final['Grupo Local de Articulação Regional'], errors='coerce')
df_final['Grupo Local de Distribuição'] = pd.to_numeric(df_final['Grupo Local de Distribuição'], errors='coerce')

# Soma das colunas
df_final['Grupo Local de Articulação Regional'] = df_final['Grupo Local de Articulação Regional'] + df_final['Grupo Local de Distribuição']

# Remove a coluna 'Grupo Local de Distribuição' após a soma
df_final.drop(columns=['Grupo Local de Distribuição'], inplace=True)
df_final['Total'] = df_final['Total'].fillna('0').astype(int)
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].fillna('0').astype(int)



# Exibe o DataFrame final
display(df_final)


In [ ]:
import pandas as pd
import numpy as np

# Dicionário de meses
meses = {
    'Jan': 'Janeiro', 'Fev': 'Fevereiro', 'Mar': 'Março', 'Abr': 'Abril', 
    'Mai': 'Maio', 'Jun': 'Junho', 'Jul': 'Julho', 'Ago': 'Agosto', 
    'Set': 'Setembro', 'Out': 'Outubro', 'Nov': 'Novembro', 'Dez': 'Dezembro'
}

df_final['sep'] = df_final['MÊS'].str.split('/').str[0]

display(df_final)

# Substituindo a abreviação pelo nome completo do mês
df_final['Período'] = np.where(df_final['sep'].isin(meses.keys()),
                           df_final['sep'].map(meses),
                           df_final['Período'])

# Exibindo o resultado
display(df_final)


In [ ]:
# Criando uma nova coluna com as três primeiras letras da coluna 'MÊS'
df_final['MÊS_abrev'] = df_final['Período'].str.slice(0, 3)

df_final['MÊS'] = df_final['MÊS_abrev'].astype(str) + '/' + df_final['particao'].astype(str)
display(df_final)

df_final.drop(columns=['sep'], inplace=True)


In [ ]:
df_final